# Part 1

Suddenly, the GPU contacts you, asking for help. Someone has asked it to simulate too many particles, and it won't be able to finish them all in time to render the next frame at this rate.

It transmits to you a buffer (your puzzle input) listing each particle in order (starting with particle 0, then particle 1, particle 2, and so on). For each particle, it provides the X, Y, and Z coordinates for the particle's position (p), velocity (v), and acceleration (a), each in the format `<X,Y,Z>`.

Each tick, all particles are updated simultaneously. A particle's properties are updated in the following order:

- Increase the X velocity by the X acceleration.
- Increase the Y velocity by the Y acceleration.
- Increase the Z velocity by the Z acceleration.
- Increase the X position by the X velocity.
- Increase the Y position by the Y velocity.
- Increase the Z position by the Z velocity.

Because of seemingly tenuous rationale involving z-buffering, the GPU would like to know which particle will stay closest to position `<0,0,0>` in the long term. Measure this using the Manhattan distance, which in this situation is simply the sum of the absolute values of a particle's X, Y, and Z position.

For example, suppose you are only given two particles, both of which stay entirely on the X-axis (for simplicity). Drawing the current states of particles 0 and 1 (in that order) with an adjacent a number line and diagram of current X positions (marked in parenthesis), the following would take place:

```
p=< 3,0,0>, v=< 2,0,0>, a=<-1,0,0>    -4 -3 -2 -1  0  1  2  3  4
p=< 4,0,0>, v=< 0,0,0>, a=<-2,0,0>                         (0)(1)

p=< 4,0,0>, v=< 1,0,0>, a=<-1,0,0>    -4 -3 -2 -1  0  1  2  3  4
p=< 2,0,0>, v=<-2,0,0>, a=<-2,0,0>                      (1)   (0)

p=< 4,0,0>, v=< 0,0,0>, a=<-1,0,0>    -4 -3 -2 -1  0  1  2  3  4
p=<-2,0,0>, v=<-4,0,0>, a=<-2,0,0>          (1)               (0)

p=< 3,0,0>, v=<-1,0,0>, a=<-1,0,0>    -4 -3 -2 -1  0  1  2  3  4
p=<-8,0,0>, v=<-6,0,0>, a=<-2,0,0>                         (0)   
```

At this point, particle 1 will never be closer to `<0,0,0>` than particle 0, and so, in the long run, particle 0 will stay closest.

Which particle will stay closest to position `<0,0,0>` in the long term?

In [42]:
import re
from io import StringIO
from textwrap import dedent
from typing import Tuple

import numpy as np
from dataclasses import dataclass

In [44]:
@dataclass
class Point:
    id_: int
    pos: Tuple[int]
    vel: Tuple[int]
    acc: Tuple[int]

In [45]:
def process_points(rows):
    points = []
    for i, row in enumerate(rows):
        p, v, a = (
            tuple(int(x) for x in item.group(0)[1:-1].split(',')) 
            for item in re.finditer(r'\<(.+?)\>', row))
        points.append(Point(i, p, v, a))
    return points

In [46]:
def _test_input():
    return process_points(StringIO(dedent("""\
        p=< 3,0,0>, v=< 2,0,0>, a=<-1,0,0>
        p=< 4,0,0>, v=< 0,0,0>, a=<-2,0,0>
        """)))
TEST_INPUT = _test_input()

In [47]:
def _puzzle_input():
    with open('./inputs/day20/input.txt') as f:
        return process_points(f)
PUZZLE_INPUT = _puzzle_input()

In [63]:
def pos_at_t(point: Point, time_arr: np.array) -> Tuple[int]:
    return tuple(point.pos[i] + (point.acc[i] * time_arr + point.vel[i]).sum() for i in range(3))

In [56]:
assert pos_at_t(TEST_INPUT[0], 1) == (4, 0, 0)
assert pos_at_t(TEST_INPUT[0], 2) == (4, 0, 0)
assert pos_at_t(TEST_INPUT[0], 3) == (3, 0, 0)

assert pos_at_t(TEST_INPUT[1], 1) == (2, 0, 0)
assert pos_at_t(TEST_INPUT[1], 2) == (-2, 0, 0)
assert pos_at_t(TEST_INPUT[1], 3) == (-8, 0, 0)

In [64]:
time_arr = np.arange(1, 100 + 1)
assert min(TEST_INPUT, key=lambda p: sum(abs(x) for x in pos_at_t(p, time_arr))).id_ == 0

In [65]:
time_arr = np.arange(1, 100000 + 1)
min(PUZZLE_INPUT, key=lambda p: sum(abs(x) for x in pos_at_t(p, time_arr)))

Point(id_=308, pos=(2978, 2082, 4280), vel=(-135, -88, -178), acc=(1, 0, 0))

# Part 2

To simplify the problem further, the GPU would like to remove any particles that collide. Particles collide if their positions ever exactly match. Because particles are updated simultaneously, more than two particles can collide at the same time and place. Once particles collide, they are removed and cannot collide with anything else after that tick.

For example:

```
p=<-6,0,0>, v=< 3,0,0>, a=< 0,0,0>    
p=<-4,0,0>, v=< 2,0,0>, a=< 0,0,0>    -6 -5 -4 -3 -2 -1  0  1  2  3
p=<-2,0,0>, v=< 1,0,0>, a=< 0,0,0>    (0)   (1)   (2)            (3)
p=< 3,0,0>, v=<-1,0,0>, a=< 0,0,0>

p=<-3,0,0>, v=< 3,0,0>, a=< 0,0,0>    
p=<-2,0,0>, v=< 2,0,0>, a=< 0,0,0>    -6 -5 -4 -3 -2 -1  0  1  2  3
p=<-1,0,0>, v=< 1,0,0>, a=< 0,0,0>             (0)(1)(2)      (3)   
p=< 2,0,0>, v=<-1,0,0>, a=< 0,0,0>

p=< 0,0,0>, v=< 3,0,0>, a=< 0,0,0>    
p=< 0,0,0>, v=< 2,0,0>, a=< 0,0,0>    -6 -5 -4 -3 -2 -1  0  1  2  3
p=< 0,0,0>, v=< 1,0,0>, a=< 0,0,0>                       X (3)      
p=< 1,0,0>, v=<-1,0,0>, a=< 0,0,0>

------destroyed by collision------    
------destroyed by collision------    -6 -5 -4 -3 -2 -1  0  1  2  3
------destroyed by collision------                      (3)         
p=< 0,0,0>, v=<-1,0,0>, a=< 0,0,0>
```

In this example, particles 0, 1, and 2 are simultaneously destroyed at the time and place marked X. On the next tick, particle 3 passes through unharmed.

How many particles are left after all collisions are resolved?

In [66]:
np.int_((1, 2, 3)).reshape((3, 1)) + 

array([[1],
       [2],
       [3]])

In [73]:
def pos_over_t(point: Point, time_arr: np.array) -> np.array:
    pos = np.int_(point.pos).reshape((3, 1))
    vel_over_t = np.empty((3, time_arr.shape[0] + 1), dtype=np.int_)
    vel_over_t[:, 0] = 0
    for i in range(3):
        vel_over_t[i, 1:] = (point.acc[i] * time_arr + point.vel[i]).cumsum()
    return pos + vel_over_t

In [76]:
pos_over_t(TEST_INPUT[1], np.arange(1, 4))

array([[ 4,  2, -2, -8],
       [ 0,  0,  0,  0],
       [ 0,  0,  0,  0]])